#这个程序作用如下：
1.初步筛选了站点（根据时间文件个数，但并没有根据缺测情况来统计)。
2.直接筛选出6h降水
3.最后输出df_final_6hourly(rain).csv

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import datetime
import pandas as pd

data_path='/home/liuli/GTS_GFS_MOS/domestic_hourly_data/df_all.csv'
file0= pd.read_csv(data_path,sep=',')
df0=pd.DataFrame(file0)

In [2]:
#print(df_all.info())
df_all=df0.drop_duplicates(subset = ['time','sta'],keep='last',inplace=False) #去掉重复的行
print(df_all)

                time       sta      ht    lat     lon       pr1      pr24  \
0         2018010100     23471    54.0  60.92   76.60  999999.0  999999.0   
1         2018010100     23552    27.0  64.92   77.82  999999.0  999999.0   
2         2018010100     23578    30.0  64.23   87.57  999999.0  999999.0   
3         2018010100     23589   105.0  64.20   93.78  999999.0  999999.0   
4         2018010100     23656    67.0  63.38   78.32  999999.0  999999.0   
5         2018010100     23657   131.0  63.12   75.28  999999.0  999999.0   
6         2018010100     23662    34.0  63.98   82.08  999999.0  999999.0   
7         2018010100     23678    45.0  63.15   87.97  999999.0  999999.0   
8         2018010100     23699   230.7  62.71  101.11  999999.0  999999.0   
9         2018010100     23741    85.0  62.75   71.67  999999.0  999999.0   
10        2018010100     23748    67.0  62.23   74.48  999999.0  999999.0   
11        2018010100     23758    68.0  62.15   77.32  999999.0  999999.0   

In [3]:
#1.先把大df中某个站点的数据挑选出来
start='2018-01-01 00:00:00'
end_of_db = '2019-01-01 00:00:00'
date_index = pd.date_range(start, end_of_db, freq = '6h')
date_df = pd.DataFrame(date_index,columns=['time'])


#把df_all的时间转换成datatime
df_all['time']=pd.to_datetime(df_all['time'],format="%Y%m%d%H")
#df_all['time'] = df_all['time'].apply(lambda x : datetime.datetime.strptime(x,'%Y%m%d%H'))
#print(df_all['time'].dtype)

#print(type(df_all['time'].dt.hour[0]))
df_qc=df_all.drop_duplicates(subset = ['sta'],keep='last',inplace=False) #申明一个不重复站点的df
print(df_qc)

/home/liuli/anaconda3/envs/MOS/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


                        time       sta      ht    lat     lon       pr1  \
21226    2018-01-01 12:00:00     A8VV3  9999.0  13.30  109.90  999999.0   
55488    2018-01-02 12:00:00     DEDG2  9999.0   4.10   92.30  999999.0   
76655    2018-01-03 03:00:00      7XJX  9999.0  29.10  129.80  999999.0   
81182    2018-01-03 07:00:00    9V9130  9999.0  12.60  112.90  999999.0   
110863   2018-01-04 05:00:00      HONU  9999.0  36.80  143.90  999999.0   
149205   2018-01-05 06:00:00     DGQD2  9999.0   5.90   89.40  999999.0   
174941   2018-01-06 01:00:00    9V5332  9999.0  37.10  143.60  999999.0   
209009   2018-01-07 00:00:00      3FOB  9999.0   5.30   91.50  999999.0   
242942   2018-01-08 00:00:00      CQYN  9999.0  50.60  178.30  999999.0   
272632   2018-01-08 22:00:00    9V3440  9999.0  54.40  176.40  999999.0   
281359   2018-01-09 05:00:00      WAZV  9999.0  46.20  151.30  999999.0   
281366   2018-01-09 05:00:00      WKPM  9999.0  25.40   80.00  999999.0   
319323   2018-01-10 07:00

In [ ]:
#df2=df_all[df_all['sta'].isin(['53603'])]
#print(df2)
#for i in df_qc.sta:  #站点不重复
#    #print(i)
#    #print(type(i))
#    df2=df_all[df_all['sta'].isin([i])] #先挑出这个站号的行 
#    df2=df2.sort_values(by='time')#把这个站点按照时间顺序排列    
#    df2=df2.reset_index(drop=True)
#    if len(df2)>4000:
#        print(df2)

In [ ]:
############开始循环读取每个站点的数据，并判断此站点的有效数据量
df_final=pd.DataFrame(columns=['time', 'sta', 'ht', 'lat', 'lon', 'pr1', 'pr24', 'pr6', 'ps', 'psl', 't2m', 'td', 'wdir', 'wspd'])

for i in df_qc.sta:  #站点不重复
    
    #print(type(i))
    df2=df_all[df_all['sta'].isin([i])] #先挑出这个站号的行
    if len(df2)>=1100: ####按照6小时一次，每天4次，一年4*365*80%（有效数据）
        print(i)
        df2=df2.sort_values(by='time')#把这个站点按照时间顺序排列    
        df2=df2.reset_index(drop=True)
        #print(df2)         
        # merge日期补充完整数据
        df2 = pd.merge(date_df,df2, on=['time'], how='left')
        df2['sta'].fillna(i,inplace=True)
        df_final=pd.concat([df_final,df2],ignore_index=True)            
    #print(df2)
    #print(df2.info())
    del df2

#这种根据某一列数值，赋值另一列适合比较简单的场景：print(df_all.loc[(float(df_all.time[-2:])%3 ==0), 'pr1'] )
df_final

45004
45005
V7ZZ5
FVNM
21638
21707
21637
30686
31513
28383
28586
28588
28688
28696
28791
29557
29625
29813
29822
29915
31913
32128
30961
30971
30978
21706
WTEE
21680
21595
21641
21530
87TEE
54287
52006
5200006
PCHM
VWTI
21708
56341
21547
21596
54748
54733
DBBE
58309
21597
91245
EUCDE19
23009
23491
23095
23460
VTLX
99217
54418
54544
54647
54720
54721
JPBN
53321
JGQH
VWXS
51149
54655
2800401
2300008
23459
57403
54583
58726
VWSM
51862
VWFQ
23992
54773
23699
48074
48095
48103
96505
98630
30074
42260
42309
42314
43413
43418
43421
43424
43436
43444
43450
43466
43467
43473
43486
43495
43497
48001
48004
48008
48010
48018
48019
48020
48025
48030
48031
48033
48035
48037
48042
48045
48048
48051
48053
48057
48060
48062
48071
48072
48075
48077
48078
48085
48087
48093
48099
48101
48107
48109
48454
48457
48463
48921
48924
48925
48926
48927
48935
48938
48941
48946
48952
48957
51158
51523
96147
42415
42623
42874
48064
48080
48094
48097
48108
48110
48112
51323
51337
54201
56114
57354
23092
2300009
23788

51811
51814
51815
51817
51818
51823
51825
51826
51827
51828
51829
51839
51844
51855
51881
51886
51894
51896
51899
51920
51931
52008
52101
52106
52112
52118
52203
52207
52211
52214
52264
52267
52313
52319
52323
52346
52378
52397
52417
52418
52424
52427
52436
52446
52447
52495
52515
52530
52532
52533
52546
52548
52557
52575
52576
52594
52602
52607
52633
52643
52645
52652
52654
52656
52657
52661
52668
52671
52674
52675
52679
52681
52707
52713
52714
52737
52745
52754
52765
52784
52787
52792
52795
52797
52814
52816
52818
52824
52825
52829
52833
52836
52838
52842
52851
52853
52854
52855
52856
52859
52862
52863
52866
52868
52869
52870
52871
52874
52875
52876
52877
52878
52881
52884
52885
52889
52895
52896
52905
52906
52908
52910
52911
52931
52941
52942
52943
52953
52955
52957
52958
52963
52968
52972
52974
52978
52980
52981
52982
52983
52984
52985
52986
52988
52993
52995
52996
52998
53068
53077
53083
53084
53149
53192
53195
53198
53225
53231
53248
53257
53261
53269
53272
53276
53282
53289
5332

In [ ]:
df_final.to_csv('./df_final_1100(rain).csv', sep=',', header=True, index=False)